<a href="https://colab.research.google.com/github/evinracher/3010090-ontological-engineering/blob/main/week2/2_06_Agent_Basico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AGENTE BASICO

In [ ]:
from google.colab import userdata
import os

# Obtener la API key desde userdata
api_key = userdata.get('GOOGLE_API_KEY')

# Opcional: Guardarla como variable de entorno
os.environ['GOOGLE_API_KEY'] = api_key

# Verificar que se haya cargado correctamente
print("API Key cargada:", "Sí" if api_key else "No")
print("Primeros caracteres:", api_key[:10] if api_key else "No encontrada")

In [ ]:
!pip install "langchain>=1,<2" langchain-core langchain-community langchain-text-splitters

In [ ]:
!pip install langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
print("✅ LangChain-Gemini instalado correctamente")

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

Definir el contexto de ejecución a proporcionar al agente y a las herramientas (tools) acceso a la base de datos.

In [ ]:
from dataclasses import dataclass

from langchain_community.utilities import SQLDatabase


# define context structure to support dependency injection
@dataclass
class RuntimeContext:
    db: SQLDatabase

Esta herramienta (tool) se conectará a la base de datos. Observe el uso de `get_runtime` para acceder al contexto de ejecución del grafo.

In [ ]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

Agrega un *system prompt* para definir el comportamiento de tus agentes.

In [ ]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows of output unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

¡Crear el agente! Añade un: model, tools, a prompt, y el acceso runtime, and vamos! Puede elegir muchos agentes de la lista de [integracion](https://docs.langchain.com/oss/python/integrations/providers/overview).

In [ ]:
import os

In [ ]:
# ========================================
# 🤖 Crear modelo Gemini
# ========================================
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",  # ✅ Modelo válido
    temperature=0.2,
)

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,  # ✅ Objeto Gemini (NO cadena)
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)



**PRIMERA CONSULTA**

In [ ]:
from langchain_core.messages import HumanMessage

question = "Which are the annual sales per year in  USA??"

inputs = {
    "messages": [HumanMessage(content=question)]
}

for step in agent.stream(
    inputs,
    context=RuntimeContext(db=db),
    stream_mode="values"
):
    msg = step.get("messages", [])[-1]
    msg.pretty_print()

**SEGUNDA CONSULTA**

In [ ]:
question = "How many attributes does the album table have??"

for step in agent.stream(
    {"messages": question},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

**TERCERA CONSULTA**

In [ ]:
from langchain_core.messages import HumanMessage

question = "Which table has the largest number of entries?"

inputs = {
    "messages": [HumanMessage(content=question)]
}

for step in agent.stream(
    inputs,
    context=RuntimeContext(db=db),
    stream_mode="values"
):
    msg = step.get("messages", [])[-1]
    msg.pretty_print()

**CUARTA CONSULTA**

In [ ]:
from langchain_core.messages import HumanMessage

question = "Obtener los 10 principales artistas por cantidad de ventas. "

inputs = {
    "messages": [HumanMessage(content=question)]
}

for step in agent.stream(
    inputs,
    context=RuntimeContext(db=db),
    stream_mode="values"
):
    msg = step.get("messages", [])[-1]
    msg.pretty_print()

**QUINTA CONSULTA**

In [ ]:
question = "Which genre on average has the longest track?"

for step in agent.stream(
    {"messages": question},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

**SEXTA CONSULTA**

In [ ]:
question = "What are the first 5 artist in the 'Artist' table? execute it"

for step in agent.stream(
    {"messages": question},
    context={"db": db},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()